# Digital signatures

In [ ]:
# Install a pip package in the current Jupyter kernel
# from: https://note.nkmk.me/en/python-pip-install-requirements/
import sys
!{sys.executable} -m pip install pycoin==0.80

In [2]:
# Adapted from: S. Nakov, "Practical Cryptography for Developers"
# https://cryptobook.nakov.com/digital-signatures/ecdsa-sign-verify-examples

# Uses ECDSA (using the elliptic curve secp256k1 + SHA3-256 hash algorithm)

from pycoin.ecdsa import generator_secp256k1, sign as signECDSA, verify as verifyECDSA
import hashlib, secrets

def sha3_256Hash(msg):
    hashBytes = hashlib.sha3_256(msg.encode("utf8")).digest()
    return int.from_bytes(hashBytes, byteorder="big")

def genKeyPair():
    privKey = secrets.randbelow(generator_secp256k1.order())
    pubKey = (generator_secp256k1 * privKey).pair()
    return [privKey, pubKey]

def sign(msg, privKey):
    msgHash = sha3_256Hash(msg)
    signature = signECDSA(generator_secp256k1, privKey, msgHash)
    return signature

def verify(msg, signature, pubKey):
    msgHash = sha3_256Hash(msg)
    valid = verifyECDSA(generator_secp256k1, pubKey, msgHash, signature)
    return valid

## Key generation

In [3]:
[privKey, pubKey] = genKeyPair()

{"privKey": hex(privKey),
 "pubKey": {
    "x": hex(pubKey[0]),
    "y": hex(pubKey[1])
 }
}


{'privKey': '0xfa77f7bece16031bf9e9575a6fad2b66698b74e782778014a718f665bc4cc28e',
 'pubKey': {'x': '0xd23675c35f7590395481fb1b216f4ed9528a2d8633f3ebb6ecee969d7b181dcb',
  'y': '0xfd5168664c68fb35d69db2218b145fbfe70ee2d47cbf29063d281b012d6123aa'}}

## Signing a message

Sign a message using the private key:

In [4]:
msg = "alice pays bob $10"

signature = sign(msg, privKey)

{"message": msg,
 "signature": {
    "r": hex(signature[0]),
    "s": hex(signature[1])
}}

{'message': 'alice pays bob $10',
 'signature': {'r': '0x4a34586cdf54836d3a4b4c7ce5fb1c61e92b73deaa2e59dd29b266a98531530a',
  's': '0xc737e56823ea4a4375c2a814c8df9e606ffe4f5287ac7aa417140de236b3a561'}}

## Verifying a signature

Verify signature of a valid signed message:

In [5]:
valid = verify(msg, signature, pubKey)

{"message": msg,
 "signature valid?": valid}

{'message': 'alice pays bob $10', 'signature valid?': True}

Verify signature of a tampered message:


In [6]:
msg2 = "alice pays bob $100"

valid = verify(msg2, signature, pubKey)

{"message": msg2,
 "signature valid?": valid}

{'message': 'alice pays bob $100', 'signature valid?': False}